# Load torch device

In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, AdamW

# --- this part is required for mac --- #
import sys, os
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '../src', '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
# comment out if youre on windows and remove 'src.' prefixes from imports #

from src.Datasets.batching import BatchManager

# Apple Silicon (M1/M2/M3/M4) optimization
# IMPORTANT: torch-scatter and torch-spline-conv may not fully support MPS!
# Enable fallback to CPU for unsupported operations
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Multi-threading optimization for M4
# Set number of threads for PyTorch operations
torch.set_num_threads(4)  # M4 has 4 performance cores
print(f"PyTorch using {torch.get_num_threads()} threads for CPU operations")

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device} (Apple Silicon GPU acceleration)")
    print(f"MPS backend available: {torch.backends.mps.is_built()}")
    print(f"⚠️  NOTE: Some PyG operations will fall back to CPU (torch-scatter, torch-spline-conv)")
    print(f"   For best M4 performance, consider using conv_type='fuse' instead of 'ori_aegnn'")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: {device}")
else:
    device = torch.device("cpu")
    print(f"Using device: {device}")
    print("⚠️  No GPU acceleration available. Training will be slow.")

PyTorch using 4 threads for CPU operations
Using device: mps (Apple Silicon GPU acceleration)
MPS backend available: True
⚠️  NOTE: Some PyG operations will fall back to CPU (torch-scatter, torch-spline-conv)
   For best M4 performance, consider using conv_type='fuse' instead of 'ori_aegnn'


# Choose NCars or NCaltech-101 Dataset

In [2]:
from src.Datasets.ncaltech101 import NCaltech
from src.Datasets.ncars import NCars

# options: 'ncars' or 'ncaltech'
#dataset = 'ncars'
dataset = 'ncaltech'

if dataset == 'ncars':
    num_classes = len(NCars.get_info().classes)
    image_size: tuple[int, int] = NCars.get_info().image_size  # (height=240, width=180)
if dataset == 'ncaltech':
    num_classes = len(NCaltech.get_info().classes)
    image_size: tuple[int, int] = NCaltech.get_info().image_size  # (height=240, width=180)

# Model Initialization

In [3]:
from src.Models.CleanEvGNN.recognition import RecognitionModel as EvGNN
from torch_geometric.data import Data as PyGData
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_shape: tuple[int, int, int] = (*image_size, 3)

# Vertausche Dimensionen: pos ist [x, y] = [width, height], also übergeben wir (width, height)
img_shape_for_model = (image_size[1], image_size[0])

evgnn = EvGNN(
    network="graph_res",
    dataset=dataset,
    num_classes = num_classes,
    img_shape=img_shape_for_model,
    dim=3,
    conv_type="fuse",
    distill=False,        # <– no KD, just normal training
).to(device)


def transform_graph(graph: PyGData) -> PyGData:
    return evgnn.data_transform(
        graph, n_samples=10000, sampling=True,
        beta=0.5e-5, radius=3.0,
        max_neighbors=16
    ).to(device)

# Dataset Initialization and processing (from the parsed dataset from the aegnn issues thread)

In [4]:
#Instantiating the ncaltech dataset
if dataset == 'ncaltech':
    dataset_obj = NCaltech(
        root=r'/Users/hannes/Documents/University/Datasets/raw_ncaltec',
        #root=r'/Users/mielgeraats/Documents/Master Artificial Intelligence/Master Project 1/Datasets/N-Caltech101',
        transform=transform_graph
    )
if dataset == 'ncars':
    dataset_obj = NCars(
        root=r'/Users/hannes/Documents/University/Datasets/raw_ncars/Prophesee_Dataset_n_cars',
        #root=r'/Users/mielgeraats/Documents/Master Artificial Intelligence/Master Project 1/Datasets/Prophesee_Dataset_n_cars',
        transform=transform_graph
)

# Processing the training part of the dataset
dataset_obj.process(modes = ["training"])

#Check how many samples are available
num_training_samples = dataset_obj.get_mode_length("training")
print(f"Number of training samples: {num_training_samples}")


x

📂 Processing folder: gerenuk


gerenuk:   0%|          | 0/27 [00:00<?, ?it/s]


📂 Processing folder: hawksbill


Error processing line 1 of /opt/anaconda3/envs/GNNBenchmark/lib/python3.11/site-packages/distutils-precedence.pth:



hawksbill:   0%|          | 0/80 [00:00<?, ?it/s]

...


📂 Processing folder: tick


tick:   0%|          | 0/39 [00:00<?, ?it/s]


📂 Processing folder: metronome


metronome:   0%|          | 0/25 [00:00<?, ?it/s]

Number of training samples: 6559


Display example events data point

In [5]:

training_set = BatchManager(
    dataset=dataset_obj,
    batch_size=64,
    mode="training"
)

# Note: BatchManager currently loads data sequentially. For faster data loading,
# consider using PyTorch's DataLoader with num_workers=4 for parallel loading.
# However, PyG's Batch.from_data_list() and transforms may not be thread-safe,
# so this requires careful implementation.

In [6]:
# Optimizer and scheduler configuration
optimizer = Adam(evgnn.parameters(), lr=5e-4, weight_decay=6e-7)

# Original scheduler (currently active)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=25, cooldown=10)

# Paper's hyperparameters and schedule (commented out - uncomment to match paper exactly)
# optimizer = Adam(evgnn.parameters(), lr=1e-3, weight_decay=5e-3)
# class LRPolicy(object):
#     def __call__(self, iteration: int):
#         # Convert iterations to approximate epochs (assuming ~230 iterations per epoch with batch=32)
#         epoch = iteration // 230
#         if epoch < 20:
#             return 1.0
#         elif epoch >= 20 and epoch < 50:
#             return 0.5
#         else:
#             return 0.1
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=LRPolicy())

loss_fn = CrossEntropyLoss()

classes = dataset_obj.get_info().classes

cls_to_idx = dict(zip(classes, range(len(classes))))

In [7]:
def label_to_index(lbl):
    if isinstance(lbl,str): # for n-caltech labels (strings)
        return cls_to_idx[lbl]
    if isinstance(lbl, torch.Tensor): # for ncars labels (tensors)
        return int(lbl.item())  

In [8]:
evgnn.train()
losses = []

# Add diagnostics for first batch
first_batch = True

# EMA for smoother loss display
ema_loss = None
ema_alpha = 0.9

print(f"Training fuse architecture for {10000} iterations to match paper performance...")
print(f"Expected result: 89-91% accuracy on NCars\n")

for i in range(1000):  # Paper trains for many more iterations (~50 epochs)
    optimizer.zero_grad()

    examples = next(training_set)
    # Use .y directly - this is how PyG batches labels
    reference = examples.y.to(device)

    # Diagnostic output for first batch
    if first_batch:
        print(f"\n=== First Batch Diagnostics ===")
        print(f"Batch size: {examples.batch.max().item() + 1}")
        print(f"Num nodes: {examples.x.shape[0]}")
        print(f"Num edges: {examples.edge_index.shape[1]}")
        print(f"Feature shape: {examples.x.shape}")
        print(f"Position range: x=[{examples.pos[:,0].min():.2f}, {examples.pos[:,0].max():.2f}], y=[{examples.pos[:,1].min():.2f}, {examples.pos[:,1].max():.2f}], t=[{examples.pos[:,2].min():.6f}, {examples.pos[:,2].max():.6f}]")
        print(f"Labels (y): {reference}")
        print(f"Label distribution: {torch.bincount(reference)}")
        if hasattr(examples, 'edge_attr') and examples.edge_attr is not None:
            print(f"Edge attr shape: {examples.edge_attr.shape}")
            print(f"Edge attr sample: {examples.edge_attr[:5]}")
        first_batch = False

    out = evgnn(examples)
    loss = loss_fn(out, reference)

    # Check for NaN/Inf
    if torch.isnan(loss) or torch.isinf(loss):
        print(f"\n!!! Loss is NaN/Inf at iteration {i} !!!")
        break

    loss.backward()

    # Gradient Clipping für Stabilität
    torch.nn.utils.clip_grad_norm_(evgnn.parameters(), max_norm=1.0)

    optimizer.step()
    scheduler.step(loss.item())  # ReduceLROnPlateau needs loss value
    # Note: If using LambdaLR (paper's schedule), change to: scheduler.step()

    # Update EMA loss for smoother display
    if ema_loss is None:
        ema_loss = loss.item()
    else:
        ema_loss = ema_alpha * ema_loss + (1 - ema_alpha) * loss.item()

    # Show predictions for first few iterations
    if i < 5 or i % 1 == 0:  # Changed to every 100 iterations
        with torch.no_grad():
            predictions = out.argmax(dim=-1)
            accuracy = (predictions == reference).float().mean().item()
            # current_lr = scheduler.get_last_lr()[0]  # Only works with LambdaLR
            print(f"Iteration {i:4d} | Loss: {loss.item():.4f} (EMA: {ema_loss:.4f}) | Acc: {accuracy:.3f}")
    elif i % 50 == 0:
        print(f"Iteration {i:4d} | Loss: {loss.item():.4f} (EMA: {ema_loss:.4f})")

    losses.append(loss.item())

print(f"\n✅ Training complete! Final EMA loss: {ema_loss:.4f}")
torch.save(evgnn.state_dict(), f"evgnn_{dataset}_fuse_test.pth")


Training fuse architecture for 10000 iterations to match paper performance...
Expected result: 89-91% accuracy on NCars


=== First Batch Diagnostics ===
Batch size: 64
Num nodes: 633378
Num edges: 8526652
Feature shape: torch.Size([633378, 1])
Position range: x=[0.00, 232.00], y=[0.00, 172.00], t=[0.000000, 0.000000]
Labels (y): tensor([ 79,   2,  77,  96,   4,   4,  46,  18,   4,  53,  37,  95,  19,  92,
         76,  57,  41,  12,  34,  53,  94,  91,  99,   4, 100,   4,  81,  42,
         31,   4,   9,  27,  72,   2,  94,  19,  72,  12,   1,  50,  11,   2,
         51,  93,   4,   3,   2,  47,   2,  29,  86,  96,  16,  15,  86,  51,
          4,   8,  99,   1,  79,  18,  96,   4])
Label distribution: tensor([0, 2, 5, 1, 9, 0, 0, 0, 1, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 2, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        2, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 0, 

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss_convergence(loss_list, window=50, title=None, xlabel="Iteration", ylabel="Loss"):
    """
    Plots the loss over iterations to visualize convergence.

    Parameters:
    - loss_list: List or array of loss values over iterations
    - title: Plot title
    - xlabel: X-axis label
    - ylabel: Y-axis label
    """
    if title is None:
        title = f"{dataset} Loss Convergence"

    plt.figure(figsize=(12,5))

    # Plot raw loss
    plt.subplot(1, 2, 1)
    plt.plot(loss_list, alpha=0.3, color='blue', label='Raw loss')
    moving_mean = np.convolve(loss_list, np.ones(window)/window, mode='valid')
    plt.plot(range(window-1, len(loss_list)), moving_mean, color='orange', linewidth=2, label=f'Moving avg (window={window})')
    plt.title(f"{title} - With Smoothing")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()

    # Plot smoothed loss only
    plt.subplot(1, 2, 2)
    plt.plot(range(window-1, len(loss_list)), moving_mean, color='orange', linewidth=2)
    plt.title(f"{title} - Smoothed Only")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

    # Print statistics
    print(f"\nLoss Statistics:")
    print(f"  Initial loss: {loss_list[0]:.4f}")
    print(f"  Final loss: {loss_list[-1]:.4f}")
    print(f"  Final smoothed loss: {moving_mean[-1]:.4f}")
    print(f"  Reduction: {(loss_list[0] - moving_mean[-1]) / loss_list[0] * 100:.1f}%")

plot_loss_convergence(losses)

# Model Evaluation on Test Set

In [ ]:
# Process test set if not already done
dataset_obj.process(modes=["test"])
num_test_samples = dataset_obj.get_mode_length("test")
print(f"Number of test samples: {num_test_samples}")

# Create test set BatchManager
test_set = BatchManager(
    dataset=dataset_obj,
    batch_size=8,
    mode="test"
)

In [ ]:
# Comprehensive test evaluation
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

evgnn.eval()

all_predictions = []
all_references = []
all_confidences = []

print("Running evaluation on test set...")
num_test_batches = num_test_samples // 32 + (1 if num_test_samples % 32 != 0 else 0)

with torch.no_grad():
    for i in range(num_test_batches):
        examples = next(test_set)
        reference = examples.y.to(device)
        out = evgnn(examples)

        # Get predictions and confidence scores
        probabilities = torch.softmax(out, dim=-1)
        prediction = out.argmax(dim=-1)
        confidence = probabilities.max(dim=-1)[0]

        # Store results
        all_predictions.extend(prediction.cpu().numpy())
        all_references.extend(reference.cpu().numpy())
        all_confidences.extend(confidence.cpu().numpy())

        if (i + 1) % 10 == 0:
            current_acc = np.mean(np.array(all_predictions) == np.array(all_references))
            print(f"Processed {i+1}/{num_test_batches} batches | Current accuracy: {current_acc:.4f}")

# Convert to numpy arrays
all_predictions = np.array(all_predictions)
all_references = np.array(all_references)
all_confidences = np.array(all_confidences)


# Test Results and Metrics

In [ ]:
# Calculate overall accuracy
accuracy = np.mean(all_predictions == all_references)
print("\n" + "="*80)
print(f"FINAL TEST RESULTS")
print("="*80)
print(f"Total test samples: {len(all_references)}")
print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Mean Confidence: {np.mean(all_confidences):.4f}")

# Per-class accuracy
print("\n" + "-"*80)
print("Per-Class Performance:")
print("-"*80)
classes = dataset_obj.get_info().classes
for class_idx, class_name in enumerate(classes):
    mask = all_references == class_idx
    if mask.sum() > 0:
        class_acc = np.mean(all_predictions[mask] == all_references[mask])
        class_count = mask.sum()
        class_conf = np.mean(all_confidences[mask])
        print(f"{class_name:15s} | Accuracy: {class_acc:.4f} | Samples: {class_count:4d} | Avg Confidence: {class_conf:.4f}")

# Classification report
print("\n" + "-"*80)
print("Detailed Classification Report:")
print("-"*80)
print(classification_report(all_references, all_predictions, target_names=classes, digits=4))


# Confusion Matrix Visualization

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(all_references, all_predictions)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes,
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - Test Set\nOverall Accuracy: {accuracy:.4f}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# Normalized confusion matrix (percentages)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=classes, yticklabels=classes,
            cbar_kws={'label': 'Percentage'})
plt.title(f'Normalized Confusion Matrix - Test Set\nOverall Accuracy: {accuracy:.4f}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()


# Confidence Distribution Analysis

In [ ]:
# Plot confidence distribution
plt.figure(figsize=(12, 5))

# Overall confidence distribution
plt.subplot(1, 2, 1)
plt.hist(all_confidences, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Prediction Confidence')
plt.ylabel('Count')
plt.title('Distribution of Prediction Confidence')
plt.grid(True, alpha=0.3)

# Confidence by correctness
plt.subplot(1, 2, 2)
correct_mask = all_predictions == all_references
correct_conf = all_confidences[correct_mask]
incorrect_conf = all_confidences[~correct_mask]

plt.hist(correct_conf, bins=30, alpha=0.7, label=f'Correct ({len(correct_conf)})', color='green', edgecolor='black')
plt.hist(incorrect_conf, bins=30, alpha=0.7, label=f'Incorrect ({len(incorrect_conf)})', color='red', edgecolor='black')
plt.xlabel('Prediction Confidence')
plt.ylabel('Count')
plt.title('Confidence Distribution by Correctness')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nConfidence Statistics:")
print(f"  Correct predictions - Mean confidence: {np.mean(correct_conf):.4f}, Median: {np.median(correct_conf):.4f}")
print(f"  Incorrect predictions - Mean confidence: {np.mean(incorrect_conf):.4f}, Median: {np.median(incorrect_conf):.4f}")


# Save Test Results

In [ ]:
# Save detailed results
results = {
    'accuracy': float(accuracy),
    'num_samples': len(all_references),
    'predictions': all_predictions.tolist(),
    'references': all_references.tolist(),
    'confidences': all_confidences.tolist(),
    'per_class_accuracy': {
        class_name: float(np.mean(all_predictions[all_references == idx] == all_references[all_references == idx]))
        for idx, class_name in enumerate(classes)
        if (all_references == idx).sum() > 0
    }
}

import json
import os

# Get the architecture type from the model
conv_type = evgnn.model.conv_type if hasattr(evgnn.model, 'conv_type') else 'fuse'

# Create results directory if it doesn't exist
results_dir = os.path.join(project_root, 'results')
os.makedirs(results_dir, exist_ok=True)

# Save with descriptive filename including dataset and architecture
results_filename = f'test_results_{dataset}_{conv_type}.json'
results_path = os.path.join(results_dir, results_filename)

with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✅ Test results saved to: {results_path}")
print(f"   Architecture: {conv_type}")
print(f"   Accuracy: {accuracy:.4f}")
